In [ ]:
!pip install openai requests pillow -q

import base64
import mimetypes
from openai import OpenAI
from IPython.display import Markdown
from google.colab import files

#API key
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

def encode_image(image_path):
    """Convert image to base64 data URI"""
    mime_type, _ = mimetypes.guess_type(image_path)
    if not mime_type or not mime_type.startswith('image'):
        mime_type = 'image/jpeg'

    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    return f"data:{mime_type};base64,{base64_image}"

#Upload blood report
uploaded = files.upload()
image_filename = next(iter(uploaded))
image_data = encode_image(image_filename)

#Enter question
your_question = input("Enter your question about the blood report: ")

#Generate response
response = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "https://colab.research.google.com",
    "X-Title": "Blood Report Analysis",
  },
  model="qwen/qwen-vl-max",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": your_question},
        {"type": "image_url", "image_url": {"url": image_data}}
      ]
    }
  ],
  max_tokens=1024
)

#Display
display(Markdown(f"### Your Question:\n{your_question}"))
display(Markdown(f"### Analysis:\n{response.choices[0].message.content}"))